In [18]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [19]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [44]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [45]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [47]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0984, -0.0566,  0.0889,  0.0924,  0.1052,  0.0263,  0.0028, -0.0471,
         -0.0064, -0.0136]], device='cuda:0', grad_fn=<AddmmBackward0>)
Predicted class: tensor([4], device='cuda:0')


In [48]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [49]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [52]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [53]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1080, -0.0959,  0.1978,  0.1573, -0.8635, -0.1770, -0.9403, -0.0171,
          0.4261, -0.1094, -0.5727,  0.0158, -0.0881,  0.5089,  0.0282,  0.2143,
          0.1619,  0.2825, -0.0664,  0.5937],
        [-0.0991, -0.3283,  0.0409,  0.2429, -0.4709, -0.2089, -0.5085, -0.1403,
          0.8241, -0.0910, -0.2388,  0.1878,  0.0804,  0.5614, -0.2330,  0.1529,
          0.0434, -0.1591,  0.0607,  0.2432],
        [-0.2123, -0.4033, -0.0020,  0.5870, -0.5255, -0.2386, -0.6133,  0.1225,
          0.3920,  0.0893, -0.5020,  0.0957, -0.0507,  0.6729, -0.1277,  0.2829,
          0.1088, -0.0764, -0.4628,  0.3914]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1080, 0.0000, 0.1978, 0.1573, 0.0000, 0.0000, 0.0000, 0.0000, 0.4261,
         0.0000, 0.0000, 0.0158, 0.0000, 0.5089, 0.0282, 0.2143, 0.1619, 0.2825,
         0.0000, 0.5937],
        [0.0000, 0.0000, 0.0409, 0.2429, 0.0000, 0.0000, 0.0000, 0.0000, 0.8241,
         0.0000, 0.0000, 0.1878, 0.0804, 0.5614, 0.00

In [55]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [56]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [57]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0133, -0.0134,  0.0258,  ...,  0.0287, -0.0044,  0.0123],
        [-0.0076, -0.0056,  0.0096,  ..., -0.0048, -0.0123, -0.0119]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0221, -0.0158], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0254,  0.0112,  0.0326,  ...,  0.0249, -0.0079, -0.0243],
        [ 0.0097, -0.0182,  0.0262,  ..., -0.0344,  0.0014,  0.0388]],
       device='cuda:0', grad_fn=<Sl